# AI6124 Assignment 4

Created by A/Prof Kai Keng ANG (kkang@i2r.a-star.edu.sg, kkang@ntu.edu.sg). Last modified 21 Oct 2024

Submitted by: [Woon Khai SHen] Mat ID: [G2304398L]

# Instructions

Enter your name according to NTU Learn, and your student ID above. Do not include the square brackets. Please save the file as POPFNN_[Student name].ipynb and submit in NTU Learn.

There are 9 questions in this assignment. Some questions have multiple parts, and the last question has 2 marks. Please read the question and hints carefully. This assignment has more open ended questions than Assignment 3. Code for some of the questions are also to be in the POPFNN class, not just below the question. So read carefully.

# Objective
In this tutorial, you will implement derivation of fuzzy if-then rules using Pseudo Outer Product (POP) Learning in POPFNN.

After completing this tutorial, you will know:

* How to apply what you leaned in the previous AI6124 assignment 3 to implement membership functions
* How to code a function to generate a given number of membership functions that span the input space of the data
* How to apply Learning Vector Quantization (LVQ), a clustering algorithm to generate interpretable membership functions from the dataset
* How to implement a simple fuzzy rule identification Pseudo Outer-Product (POP) learning algorithm to identify if-then-fuzzy rules using the generated membership functions and the dataset
* How to optimize the if-then-fuzzy rules identified

Prerequisites:
1. Python programming. https://docs.python.org/3/tutorial/.
2. Completion and understanding of AI6124 Assignment 3
3. Fuzzylab
4. AI6124 Lectures 9, 10, 11

Feel free to edit and use some code from this notebook for your group project, but please remove the assignment questions and your answers.

# Introduction

First we install the Fuzzy Logic libraries, and import the necessary libraries.

In [ ]:
!pip install -U fuzzylab
import numpy as np
import matplotlib.pyplot as plt
from random import seed
import sys
import fuzzylab as fz


# 1. Fuzzy Membership and NFS basics

In order to use Fuzzy Membership functions and a Neuro-Fuzzy System such as POPFNN, we need to create the basic functions and classes to store the parameters.

In this tutorial, we will use the Guassian 2mf membership function. This function has 2 parameters more than the Gaussian membership function. Both are provided for reference. An evalmf function is also included to call the respective function to evaluate a membership output.

In [ ]:
def gaussmf(x, params):
    assert len(params) == 2, 'Gaussmf function must have 2 parameters.'
    sig, c = np.asarray(params)
    return np.exp(-pow((x - c), 2) / (2 * pow(sig, 2)))

def gauss2mf(x, params):
    assert len(params) == 4, 'Gauss2mf function must have 4 parameters.'
    sig1, c1, sig2, c2 = np.asarray(params)
    assert c1 <= c2, 'c1 <= c2 is required.'
    if np.isscalar(x):
        if x<=c1:
            y=gaussmf(x, [sig1, c1])
        else:
            y=gaussmf(x,[sig2, c2])
    else:
        y = np.ones(len(x))
        idx1 = (x <= c1)
        idx2 = (x > c2)
        y[idx1] = gaussmf(x[idx1], [sig1, c1])
        y[idx2] = gaussmf(x[idx2], [sig2, c2])
    return y

# This function overwrites the evalmf function of the fuzzy logic toolbox by looking for the function name defined by
# mf.Type and calling it directly.
def evalmf(mf, x):
    possibles = globals().copy()
    possibles.update(locals())
    method = possibles.get(mf.Type)
    return method(x, mf.Parameters)

## Fuzzy Membership Function and POPFNN classes

Next we create a Fuzzy Membership function class to store the Fuzzy Membership function created, and a POPFNN class to store the input, output and fuzzy rules created.

In [ ]:
class fuzzymf(object):
    def __init__(self, Type, Parameters):
        self.Type = Type
        self.Parameters = Parameters
    def __repr__(self):
            return 'fismf, '\
                ' Type: %s, '\
                ' Parameters: %s\n'\
                % (self.Type,self.Parameters)

class popfnn(object):
    In_mf:fuzzymf
    Out_mf:fuzzymf
    N_inputs:int
    N_outlabels:int
    Lut_m:np
    Lut_d:np
    N_rules:int
    Pweights:np

    def __init__(self, In_mf, Out_mf):
        self.In_mf = In_mf
        self.Out_mf = Out_mf

        self.N_inputs=len(self.In_mf)
        self.N_outlabels=len(self.Out_mf[0])
        self.Lut_m=np.empty(self.N_inputs, np.int8)
        self.Lut_d=np.empty(self.N_inputs, np.int8)

        # Calculate lookup tables for rule access
        self.N_rules = 1
        for i in range(self.N_inputs):
            self.Lut_m[i] = len(self.In_mf[i]) #n_mf
            self.Lut_d[i] = 1
            self.N_rules = self.N_rules * self.Lut_m[0]
            for j in range(i):
                self.Lut_d[j] = self.Lut_d[j] * self.Lut_m[j]

        # Initialize pseudo weights for all rules
        self.pweights = np.zeros([self.N_rules,self.N_outlabels])

    # returns the membership label given rule number and the ninput
    def getlabel(self, rule, ninput):
        return ( int((int(rule)/(self.Lut_d[ninput]))%self.Lut_m[ninput]));

    # prints all the rules
    def printruleslabels(self):
        print('N_inputs=',self.N_inputs)
        print('N_rules=',self.N_rules)
        print('N_output_mf=',self.N_outlabels)
        print('rule_number, labels, pweights')
        for i in range(self.N_rules):
            print(i,end = ' ')
            for j in range(self.N_inputs):
                print(self.getlabel(i,j),end = ' ')
            for j in range(self.N_outlabels):
                print(self.pweights[i][j],end = ' ')
            print(' ')

    # Enter code for question 8 here for print_int_labels

    def poplearn(self, x, y):
        for r in range(self.N_rules):
            min_t=1.0
            for i in range(self.N_inputs):
                # Get the label of input i for rule r
                label=self.getlabel(r,i)
                # Compute membership of input i
                t=evalmf(self.In_mf[i][label],x[i])
                # Compute min across all input membership functions
                min_t=min(min_t,t)
            for i in range(self.N_outlabels):
                t=evalmf(self.Out_mf[0][i],y)
                self.pweights[r][i]+=min_t*t

    # Enter code for question 5 here for function identify_rules

    # Enter code for question 9 here for function poppredict(self, x) and return predicted y


# 2. Generating Fuzzy Membership functions by spanning the input space

Next we create a simple function to generate a given number of membership functions that span the input space of the data.

In [ ]:
# This function creates nlabels number of gauss2mf that spans min and max of the inputs
def span_learnmem(x, params):

    assert len(params) == 2, 'spam_learnmem function must have 2 parameters.'
    nlabels, width = np.asarray(params)
    nlabels=int(nlabels)

    # First get the max and min of each dimension
    maxx=np.amax(x, axis=0)
    minx=np.amin(x, axis=0)

    if x.ndim==1:
        ninputs=1
    else:
        ninputs=np.size(x,1)

    fis=[]
    for i in range(ninputs):
        if ninputs==1:
            centroids=np.linspace(minx,maxx,int(nlabels))
        else:
            centroids=np.linspace(minx[i],maxx[i],int(nlabels))
        sig=(centroids[1]-centroids[0])*width
        mf=[]
        for j in range(nlabels):
            mf.append(fuzzymf(Type = 'gauss2mf', Parameters = [sig, centroids[j], sig, centroids[j]]))
        fis.append(mf)
    return fis


# 3. Generate Training dataset

Now we generate a dataset to train the Neuro Fuzzy System.

In [ ]:
seed(1)
dataset = np.random.randn(200,2)
dataset[0:100]+=(+2)
dataset[101:200]+=(-2)
labels=np.array([1]*100)
labels=np.append(labels,[2]*100)
#labels=labels.transpose()
plt.scatter(dataset[labels==1,0], dataset[labels==1,1])
plt.scatter(dataset[labels==2,0], dataset[labels==2,1])
plt.legend(['y=1','y=2'])
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')

# 4. Train the fuzzy membership functions

Now we use the span_learnmem function to generate the fuzzy membership functions. After POPFNN initializes the rule space based on the input and output membership functions. The function printrulelabels prints out all the rules, the membership function labels for each input, and the pseudo weights of each rule.

In [ ]:
inmf = span_learnmem(dataset, [3, 0.5])
outmf = span_learnmem(labels, [3, 0.5])
nf=popfnn(In_mf=inmf, Out_mf=outmf)
nf.printruleslabels()

<font color=red>Question 1(a)</font>: How many rules are initialized in the above code? <font color='red'>(0.5 mark)</font>

Enter your answer to Question 1(a) here.
Number of rules initialized =


<font color=red>Question 1(b)</font>: How many rules are generated if you change span_learnmem to generate 2 membership functions instead of 3 for both input and output membership functions? <font color=blue>Hint: Examine the code carefully. </font><font color='red'>(0.5 mark)</font>

In [ ]:
# Modify code for question 1(b) here
inmf2 = span_learnmem(dataset, [3, 0.5])
outmf2 = span_learnmem(labels, [2, 0.5])
nf2=popfnn(In_mf=inmf, Out_mf=outmf)
nf2.printruleslabels()

Enter your answer to Question 1(b) here. Number of rules initialized =


In [ ]:
x = np.linspace(-15, 10, 101)
plt.plot(x, evalmf(inmf[0][0], x))
plt.plot(x, evalmf(inmf[0][1], x))
plt.xlabel('$x_1$')
plt.ylabel('$\mu(x_1)$')
plt.title('Membership Functions for Input $x_1$')

In [ ]:
plt.plot(x, evalmf(inmf[1][0], x))
plt.plot(x, evalmf(inmf[1][1], x))
plt.xlabel('$x_2$')
plt.ylabel('$\mu(x_2)$')
plt.title('Membership Functions for Input $x_2$')

In [ ]:
y = np.linspace(0, 2.5, 101)
plt.plot(y, evalmf(outmf[0][0], y))
plt.plot(y, evalmf(outmf[0][1], y))
plt.xlabel('$y_1$')
plt.ylabel('$\mu(y)$')
plt.title('Membership Functions for Output $y_1$')

<font color=red>Question 2(a)</font>: Modify the span_learnmem to include a parameter called tail. If parameter is true, then all the membership functions generated will cover the universe of discourse. <font color=blue>Hint: One end of the left most and the other end of the right most membership function will give a membership value of 1, but not the middle of the membership function. </font><font color='red'>(0.5 mark)</font>

In [ ]:
# Modify the below code here for Question 2(a).
def span_learnmem1(x, params):

    assert len(params) == 3, 'spam_learnmem1 function must have 3 parameters.'
    nlabels, width, tail = np.asarray(params)
    nlabels=int(nlabels)
    tail=bool(tail)

    # First get the max and min of each dimension
    maxx=np.amax(x, axis=0)
    minx=np.amin(x, axis=0)

    if x.ndim==1:
        ninputs=1
    else:
        ninputs=np.size(x,1)

    for i in range(ninputs):
        if ninputs==1:
            centroids=np.linspace(minx,maxx,int(nlabels))
        else:
            centroids=np.linspace(minx[i],maxx[i],int(nlabels))
        sig=(centroids[1]-centroids[0])*width
        mf=[]
        for j in range(nlabels):
            mf.append(fismf(Type = 'gauss2mf', Parameters = [sig, centroids[j], sig, centroids[j]]))
        fis.append(mf)
    return fis


<font color=red>Question 2(b)</font>: Use the above modified span_learnmem and plot nlabels=5 membership functions with tail=true for $x_2$ <font color='red'>(0.5 mark)</font>

In [ ]:
# Enter code here for Question 2(b).

<font color=red>Question 3(a)</font>: Now modify span_learnmem2 below without the width parameter and automatically compute the width such that the membership functions intersect at value $\mu=0.5$ between the centroids. Retain the tail parameter. <font color='red'>(0.5 mark)</font>

In [ ]:
# This function creates nlabels number of gauss2mf that spans min and max of the inputs
def span_learnmem2(x, params):

    assert len(params) == 2, 'spam_learnmem function must have 2 parameters.'
    nlabels, tail = np.asarray(params)
    nlabels=int(nlabels)
    tail=bool(tail)

    # First get the max and min of each dimension
    maxx=np.amax(x, axis=0)
    minx=np.amin(x, axis=0)

    if x.ndim==1:
        ninputs=1
    else:
        ninputs=np.size(x,1)

    for i in range(ninputs):
        if ninputs==1:
            centroids=np.linspace(minx,maxx,int(nlabels))
        else:
            centroids=np.linspace(minx[i],maxx[i],int(nlabels))
        sig=(centroids[1]-centroids[0])*width
        mf=[]
        for j in range(nlabels):
            mf.append(fismf(Type = 'gauss2mf', Parameters = [sig, centroids[j], sig, centroids[j]]))
        fis.append(mf)
    return fis

<font color=red>Question 3(b)</font>: Use the above modified span_learnmem and plot nlabels=3 membership functions with tail=true for $x_1$ <font color='red'>(0.5 mark)</font>

In [ ]:
# Enter code here for Question 3(b).

<font color=red>Question 4</font>: Now create a new LVQ_learnmem with nlabels and tail parameters. Then plot nlabels=2 membership functions with tail=true for $x_2$. You can refer to code from AI6124 Assignment 3. Also automatically compute the width such that the membership functions intersect at value $\mu=0.5$ between the centroids<font color='red'> (1 mark)</font>

In [ ]:
# Enter code here for Question 4.

# 5. Train the POPFNN using POP learning

Next we train the POPFNN Neuro-Fuzzy System using Pseudo Outer-Product Learning algorithm. Becareful that you do not change some of the codes along the way. Check to ensure this code below produce only 4 if-then fuzzy rules before continue with the rest of the questions.

In [ ]:
# Perform POP learning
inmf = span_learnmem(dataset, [2, 0.5])
outmf = span_learnmem(labels, [2, 0.5])
nf=popfnn(In_mf=inmf, Out_mf=outmf)
for k in range(len(dataset)):
    nf.poplearn(dataset[k],labels[k])
nf.printruleslabels()

<font color=red>Question 5</font>: Examine the pseudo weights (pweights) for each output label in the output above. At the end of POP learning, each pweight corresponds to the respective outputs. Create a function in POPFNN class call identify_rules to identify the consequent for each rule based on the value of the pweights. <font color=blue>Hint: You just need to print the rule_number and the consequent. </font><font color='red'>(1 mark)</font>

In [ ]:
# Modify code in POPFNN class for Question 5
# Code to print rules for Question 5
nf.identify_rules()

<font color=red>Question 6</font>: Now create a function call print_int_labels in POPFNN class to print fuzzy if-then rules in an interpretable form with the consequent identified from question 5 above. <font color=blue>Hint: The if-then fuzzy rule printed should be in the form: "Rule 0: If x1 is small and x2 is large Then y is blue".</font><font color='red'>(1 mark)</font>  

In [ ]:
# Code to print interpretable rules for Question 8.
nf.print_int_labels()

<font color=red>Question 7</font>: Now change the generation of membership function to LVQ with nlabels=2 and tail=true. Retrain the fuzzy rules and print the rules for inspection. Any changes in the interpretable rules identified? <font color='red'>(1 mark)</font>

In [ ]:
# Enter code for Question 7

# Code to print interpretable rules for Question 7
nf.print_int_labels()

Enter your answer to Question 7 here. [Answer Yes or No]

<font color=red>Question 8</font>: Now inspect the pseudo weights again. Some of the rules have pseudo weights that are very similar for both output labels. Implement another function to identify the more significant rules such that the pseduo weight for a particular output label is much higher than the other output labels. <font color=blue>Hint: How significant the rule is depends on a parameter that can be specified.You can specify this parameter. Examine the pweights carefully. Hints will also be mentioned in lecture.</font><font color='red'>(1 mark)</font>

In [ ]:
# Enter code for Question 8.

<font color=red>Question 9</font>: Now implement a function in POPFNN class to predict the label y using the learned if-then-fuzzy rules. Use the optimized rules after question 8 above. However, if you are unable to answer question 8 above, you can use the unoptimized rules, but you have to indicate so in the answers.<font color=blue>Hint: You have to implement Compositional Rule of Inference (CRI) using the inputs values, the input membership function, and the output membership function. For defuzzification, you can use the Centre of Area for  Gaussian output membership function, which is just the Singleton value for the centroid of the output membership function.</font><font color='red'> (2 marks)</font>

In [ ]:
# Code for Question 9
seed(1234)
testdataset = np.random.randn(4,2)
testdataset[0:2]+=(+2)
testdataset[3:4]+=(-2)
print(testdataset)
predicted_y=nf.poppredict(testdataset)
print(predicted_y)